In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
N = 1000

In [3]:
ip = np.random.rand(N) > 0.5
ip = ip.astype(int)

In [4]:
Es = 1

In [5]:
eb_no_db = np.arange(0,30)

In [6]:
capacity_rate_CSIT = np.zeros(len(eb_no_db))

In [7]:
def sigma_four(noise_variance,S1,S2,S3,S4,q_optimal_3):
    
    S = [S1, S2, S3, S4]
    
    q_optimal_3 = q_optimal_3
    
    sigma_four_q = []
    
    summation_part = 0
    
    for kk in range(len(S)):
        
        summation_part = summation_part + (noise_variance/np.square(S[kk]))
        
    mu = (1/len(S)) * (Es + summation_part)
    
    for mm in range(len(S)):
        
        qi = mu - (noise_variance/np.square(S[mm]))
        
        if(qi > 0 ):
            
            sigma_four_q.append(qi)
            
        else:
            sigma_four_q = []
            
            sigma_four_q = q_optimal_3.copy()
            
            break
            
    return sigma_four_q

In [8]:
def sigma_three(noise_variance,S1,S2,S3,q_optimal_2):
    
    S = [S1, S2, S3]
    
    q_optimal_2 = q_optimal_2
    
    sigma_three_q = []
    
    summation_part = 0
    
    for kk in range(len(S)):
        
        summation_part = summation_part + (noise_variance/np.square(S[kk]))
        
    mu = (1/len(S)) * (Es + summation_part)
    
    for mm in range(len(S)):
        
        qi = mu - (noise_variance/np.square(S[mm]))
        
        if(qi > 0 ):
            
            sigma_three_q.append(qi)
            
        else:
            sigma_three_q = []
            
            sigma_three_q = q_optimal_2.copy() 
            
            break
            
    return sigma_three_q

In [9]:
def sigma_two(noise_variance,S1,S2,q_optimal_1):
    
    S = [ S1 , S2 ]
    
    q_optimal_1 = q_optimal_1

    sigma_two_q  = []
    
    summation_part = 0
    
    for kk in range(len(S)):
        
        summation_part = summation_part +  (noise_variance/np.square(S[kk]))
        
    mu = (1/len(S))* (Es + summation_part)
    
    for mm in range(len(S)):
        
        qi = mu - (noise_variance/np.square(S[mm]))
        
        if(qi > 0):
            
            sigma_two_q.append(qi)
            
        else:
            sigma_two_q = []
            
            sigma_two_q = q_optimal_1.copy()
            
            break
            
    return sigma_two_q

In [10]:
def sigma_one(noise_variance,S1):
    
    
    S = [S1]
    
    
    sigma_one_q = []
    
    summation_part = noise_variance / np.square(S1)
    
    mu = (1/len(S)) * (Es + summation_part )
    
    q_optimal = mu - (noise_variance / np.square(S1))
    
    if q_optimal > 0 :
        
        sigma_one_q.append(q_optimal)
        
    
    
    return sigma_one_q

In [11]:
def diagonalQ(noise_variance,S):
    
    S1 = S[0]
    
    S2 = S[1]
    
    S3 = S[2]
    
    S4 = S[3]
    
    S = [S1, S2, S3, S4]
    
    q_optimal_1 = sigma_one(noise_variance,S1)
    
    q_optimal_2 = sigma_two(noise_variance,S1,S2,q_optimal_1)
    
    q_optimal_3 = sigma_three(noise_variance,S1,S2,S3,q_optimal_2)
    
    q_optimal_4 = sigma_four(noise_variance,S1,S2,S3,S4,q_optimal_3)
    
    return q_optimal_4
    
     

In [12]:
def csit_Rs():
    
    for jj in range(len(eb_no_db)):
        
        snr = 10**(eb_no_db[jj] / 10)
        
        nTx = 4
        
        nRx = 4
        
        Rz = (1/snr) * np.eye(nRx)
        
        Rz_inverse = np.linalg.inv(Rz )
        
        noise_variance = (1/snr)
        
        total_capacity = 0
        
        for ii in range(N):
            
            H = (1/np.sqrt(2)) * (np.random.randn(nTx,nRx) + 1j * np.random.randn(nTx,nRx))
            
            H_hermitian = np.conjugate(H)
            
            H_hermitian = np.transpose(H_hermitian)
            
            U,S,Vh = np.linalg.svd(H)
            
            V = np.conjugate(Vh)
            
            V = np.transpose(V)
            
            diagonal_q = diagonalQ(noise_variance,S)
            
            diagonal_matrix = np.diag(diagonal_q)
            
            if diagonal_matrix.shape[0] < 4 or diagonal_matrix.shape[1] < 4:
                
                diagonal_matrix = np.pad(diagonal_matrix, ((0, 4 - diagonal_matrix.shape[0]), (0, 4 - diagonal_matrix.shape[1])), mode='constant')
            
            Rs = V @ diagonal_matrix @ Vh
            
            z = (np.eye(nRx) + Rz_inverse @ H @ Rs @ H_hermitian)
            
            c = np.linalg.det(z)
            c = np.log2(abs(c))
            
            total_capacity = total_capacity + c
            
        capacity_rate_CSIT[jj] = total_capacity / N
    
    return capacity_rate_CSIT
            
        
            

In [13]:
csit_Rs()

array([ 4.20327345,  4.74949168,  5.37083871,  5.95321918,  6.59585399,
        7.34723205,  8.10268502,  8.9228246 ,  9.76681611, 10.66684771,
       11.52142855, 12.40202536, 13.46631543, 14.48855373, 15.45674641,
       16.54442206, 17.65610013, 18.87131674, 19.86714019, 20.99404322,
       22.22722069, 23.38667931, 24.65265332, 25.94763279, 27.16492971,
       28.54002089, 29.69659499, 31.04169933, 32.45267944, 33.71245778])